In [2]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [3]:
# 数据增强程序
# -----Type
#    |-----Type_1
#         |- 1.jpg
#         |- 2.jpg
#        ...
#    |-----Type_2
#    |-----Type_3
!mkdir -p drive
!google-drive-ocamlfuse drive
import os
os.chdir("drive/flower_data") 
! ls

drive  model_torch.pkl	train  Untitled0.ipynb	valid


In [0]:
import torch
import torch.nn.functional as F
from torch import nn
from torch import optim
from torchvision import datasets, transforms, models

from PIL import Image

import numpy as np

import json

import matplotlib.pyplot as plt

In [0]:
data_dir = '../forhsil'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/val'
# test_dir = data_dir + '/test'

In [0]:
# training data: random scaling, cropping, flipping, resized data to 224x224
train_transforms = transforms.Compose([# transforms.RandomRotation(30),
                    transforms.Resize([224,224]),
                    # transforms.RandomResizedCrop(224),
                    # transforms.RandomHorizontalFlip(),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

# testing and validation: resize and crop to the appropriate size
test_transforms = transforms.Compose([transforms.Resize([255,255]),
                    # transforms.Resize(224),
                    transforms.CenterCrop(224),
                    transforms.ToTensor(),
                    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

In [0]:
# Load the datasets with ImageFolder
train_data = datasets.ImageFolder(train_dir, transform=train_transforms)
# test_data = datasets.ImageFolder(test_dir, transform=test_transforms)
valid_data = datasets.ImageFolder(valid_dir, transform=test_transforms) # use test_transforms

# Define the dataloaders
trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
# testloader = torch.utils.data.DataLoader(test_data, batch_size=32)
validloader = torch.utils.data.DataLoader(valid_data, batch_size=32)

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# model = models.vgg16(pretrained=True)
model = models.resnet101(pretrained=True)
for param in model.parameters():
    param.requires_grad = False
print(model)

cpu
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bi

In [0]:
'''
model.classifier = nn.Sequential(               nn.Linear(25088, 512),
                                 nn.ReLU(),
                                 nn.Dropout(0.05),
                                 nn.Linear(512, 50),
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)
model.to(device)
'''

In [29]:
model.fc = nn.Sequential(               nn.Linear(2048, 512),
                                 nn.ReLU(),
                                 # nn.Dropout(0.05),
                                 nn.BatchNorm1d(512),
                                 nn.Linear(512, 2),
                                 nn.LogSoftmax(dim=1))
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=F

In [0]:
# Define variables we use for the training
# Set epochs
epochs = 15
# Number of train steps, so we track it
steps = 0
# Track the loss, so set to zero
running_loss = 0
# Steps we want to go before print out the validation loss
print_every = 5
accuracy_train = 0

In [31]:
# Training, looping through the epochs
train_acc = []
train_loss = []
val_acc = []
val_loss = []
for epoch in range(epochs):   
    # looping through the data to train       
    for inputs, labels in trainloader:       
        steps += 1
        # Move input and label tensors to the default device
        inputs, labels = inputs.to(device), labels.to(device)
        # zero our gradients
        optimizer.zero_grad() 
        # Get our log probabilties from our model
        logps = model.forward(inputs).to(device)
        ps = torch.exp(logps)
        top_p, top_class = ps.topk(1, dim=1)

        equals = top_class == labels.view(*top_class.shape)
        accuracy_train += torch.mean(equals.type(torch.FloatTensor)).item()
        # Get loss from the criterion on the labels
        loss = criterion(logps, labels) 
        # Backward pass
        loss.backward(retain_graph=True) 
        # Take a step with the optimizer
        optimizer.step() 
        # keep track of our training loss
        running_loss += loss.item()
        # if zero, we go into the validation loop
        if steps % print_every == 0:
            valid_loss = 0
            accuracy = 0
            # Turn the model into evaluation inference mode which turns off dropout
            # so we can use the network to make predictions instead a test loss and accuracy
            model.eval()
            with torch.no_grad():
                # Get images and label from the validation data
                for inputs, labels in validloader:
                    # Transfer tensors over to the GPU
                    inputs, labels = inputs.to(device), labels.to(device)
                    logps = model.forward(inputs).to(device)
                    batch_loss = criterion(logps, labels)
                    # Keep track of our loss to test 
                    valid_loss += batch_loss.item()
                    
                    # Calculate accuracy
                    ps = torch.exp(logps)
                    top_p, top_class = ps.topk(1, dim=1)
                    equals = top_class == labels.view(*top_class.shape)
                    accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
            
            train_acc.append(accuracy_train/print_every)      
            train_loss.append(running_loss/print_every) 
            val_acc.append(accuracy/len(validloader)) 
            val_loss.append(valid_loss/len(validloader)) 
            with open('hsil_train_loss.txt', 'w') as f:
                b = [f.write(str(i) + '\n') for i in train_loss]

            with open('hsil_train_accur.txt', 'w') as ac:
                b = [ac.write(str(i) + '\n') for i in train_acc]

            with open('hsil_test_accur.txt', 'w') as af:
                b = [af.write(str(i) + '\n') for i in val_acc]

            with open('hsil_test_loss.txt', 'w') as af:
                b = [af.write(str(i) + '\n') for i in val_loss]
            
            print(f"Epoch {epoch+1}/{epochs}.. "
               f"Train loss: {running_loss/print_every:.3f}.. "
               f"Train acc: {accuracy_train/print_every:.3f}.. "
               f"Validation loss: {valid_loss/len(validloader):.3f}.. "
               f"Validation accuracy: {accuracy/len(validloader):.3f}")
            
            # Reinit running loss
            running_loss = 0
            accuracy_train = 0
            # Put the model back into training mode
            model.train()

Epoch 1/15.. Train loss: 0.531.. Train acc: 0.688.. Validation loss: 0.689.. Validation accuracy: 0.500
Epoch 1/15.. Train loss: 0.329.. Train acc: 0.863.. Validation loss: 0.541.. Validation accuracy: 0.740
Epoch 1/15.. Train loss: 0.460.. Train acc: 0.819.. Validation loss: 0.396.. Validation accuracy: 0.832
Epoch 1/15.. Train loss: 0.374.. Train acc: 0.812.. Validation loss: 0.337.. Validation accuracy: 0.865
Epoch 1/15.. Train loss: 0.419.. Train acc: 0.812.. Validation loss: 0.361.. Validation accuracy: 0.829
Epoch 1/15.. Train loss: 0.358.. Train acc: 0.856.. Validation loss: 0.352.. Validation accuracy: 0.827
Epoch 1/15.. Train loss: 0.410.. Train acc: 0.800.. Validation loss: 0.332.. Validation accuracy: 0.858
Epoch 1/15.. Train loss: 0.274.. Train acc: 0.881.. Validation loss: 0.356.. Validation accuracy: 0.849
Epoch 1/15.. Train loss: 0.292.. Train acc: 0.881.. Validation loss: 0.374.. Validation accuracy: 0.846
Epoch 1/15.. Train loss: 0.360.. Train acc: 0.838.. Validation l

KeyboardInterrupt: ignored

In [0]:
torch.save(model, 'model_torch.pkl')